# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726671e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Train the agent

In [ ]:
from ddpg_agent import Agent
from collections import deque, Counter
import torch
import matplotlib.pyplot as plt
%matplotlib inline
from workspace_utils import active_session
import datetime

agent = Agent(state_size = state_size, action_size = action_size, random_seed=2,
              actor_fc1_units=100, actor_fc2_units=100, critic_fc1_units=100, critic_fc2_units=100,
              lr_actor=1e-5, lr_critic=1e-4, buffer_size=int(1e6))

# Resume from previous training session.
# agent.actor_local.load_state_dict(torch.load('checkpoint_actor_continue.pth'))
# agent.actor_target.load_state_dict(torch.load('checkpoint_actor_continue.pth'))
# agent.critic_local.load_state_dict(torch.load('checkpoint_critic_continue.pth'))
# agent.critic_target.load_state_dict(torch.load('checkpoint_critic_continue.pth'))


def ddpg(n_episodes=6000, max_t=1000, print_every=50):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] 
        state = env_info.vector_observations
        agent.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state)
            # am i passing in action correctly?
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            print(datetime.datetime.now())
    
    return scores

with active_session():
    print("Start: {}".format(datetime.datetime.now()))
    scores = ddpg()

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, len(scores)+1), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()
    print("End: {}".format(datetime.datetime.now()))

Start: 2018-11-04 23:08:04.760634
Episode 50	Average Score: 11.80
2018-11-04 23:17:38.791927
Episode 100	Average Score: 14.25
2018-11-04 23:27:42.957003
Episode 150	Average Score: 15.15
2018-11-04 23:38:10.746697
Episode 200	Average Score: 17.17
2018-11-04 23:49:26.958149
Episode 250	Average Score: 19.25
2018-11-05 00:01:38.949255
Episode 300	Average Score: 20.04
2018-11-05 00:14:53.384714
Episode 350	Average Score: 19.76
2018-11-05 00:29:21.057841
Episode 400	Average Score: 20.58
2018-11-05 00:44:55.719724
Episode 450	Average Score: 22.79
2018-11-05 01:01:22.619214
Episode 500	Average Score: 25.18
2018-11-05 01:19:20.826943
Episode 550	Average Score: 24.40
2018-11-05 01:38:56.942889
Episode 600	Average Score: 26.18
2018-11-05 01:59:28.862787
Episode 650	Average Score: 30.12
2018-11-05 02:20:51.287765
Episode 700	Average Score: 33.36
2018-11-05 02:42:58.257922
Episode 750	Average Score: 33.27
2018-11-05 03:05:52.670894
Episode 775	Average Score: 33.84